In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("Food_Inspections.csv")

In [99]:
def parse_violations(df):
    
    dfv = df[["Inspection ID","License #","Violations","Results"]]
    dfv = dfv.dropna()
    split_v = dfv['Violations'].str.split("|",expand=True)
    
    for c in split_v.columns:
        colname = "Violation_"+str(c)
        dfv[colname] = split_v[split_v.columns[c]]
    dfv = dfv.drop(columns='Violations')
    
    ID_col = dfv.columns.get_loc("Inspection ID")
    Lic_col = dfv.columns.get_loc("License #")
    res_col = dfv.columns.get_loc("Results")
    Vio_strt_col = dfv.columns.get_loc("Violation_0")
    
    rows={"Inspection ID":[], "License #":[], "Results":[], "Violation":[]}

    for x in range(0,len(dfv)):
        insp, lic, res = dfv.iat[x,ID_col],dfv.iat[x,Lic_col], dfv.iat[x,res_col]
        violations = dfv.iloc[x,Vio_strt_col:list(np.where(np.array(dfv.iloc[x,Vio_strt_col:])!=None)[0]+4)[-1]]
        
        for v in violations:
            rows['Inspection ID'].append(insp)
            rows['License #'].append(lic)
            rows['Results'].append(res)
            rows['Violation'].append(v)
            
        if x%10000==0:
            print(x, "/", len(dfv))
            
    
    dff = pd.DataFrame(rows)
    vios = dff['Violation'].str.split(".", n=1, expand=True)
    dff = dff.drop(columns='Violation')
    dff['Violation #']=vios[0].apply(lambda x: int(x.strip()))
    dff['Violation Desc']=vios[1].str.split("- Comments:", n=1, expand=True)[0].apply(lambda x: x.strip())
    dff['Comments']=vios[1].str.split("- Comments:", n=1, expand=True)[1].apply(lambda x: x.strip())
    
    return dff

In [100]:
dff = parse_violations(df)

0 / 123002
10000 / 123002
20000 / 123002
30000 / 123002
40000 / 123002
50000 / 123002
60000 / 123002
70000 / 123002
80000 / 123002
90000 / 123002
100000 / 123002
110000 / 123002
120000 / 123002


In [101]:
dff

,Inspection ID,License #,Results,Violation #,Violation Desc,Comments
0,2079125,2446638.0,Not Ready,8,SANITIZING RINSE FOR EQUIPMENT AND UTENSILS: ...,"NO DISH WASHING FACILITIES ON SITE, (NO THREE ..."
1,2079125,2446638.0,Not Ready,11,"ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, DESIG...","NO EXPOSED HAND SINK FOR REAR SERVICE AREA, IN..."
2,2079125,2446638.0,Not Ready,18,NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS...,NO LICENSE PEST CONTROL LOG BOOK AT THIS TIME ...
3,2079123,2517338.0,Pass,35,"WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTE...",MUST PROVIDE OVERHEAD PROTECTION FOR COFFEE/TE...
4,2079123,2517338.0,Pass,34,"FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD RE...",FLOORS AT COFFEE/TEA STATIONS MUST BE MADE SMO...
...,...,...,...,...,...,...
568613,104236,80916.0,Fail,36,LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF LIG...,"All rooms in which food or drink is prepared, ..."
568614,104236,80916.0,Fail,37,TOILET ROOM DOORS SELF CLOSING: DRESSING ROOMS...,None of the operations connected with food est...
568615,104236,80916.0,Fail,38,VENTILATION: ROOMS AND EQUIPMENT VENTED AS REQ...,The flow of air discharged from kitchen fans s...
568616,104236,80916.0,Fail,39,LINEN: CLEAN AND SOILED PROPERLY STORED,"Soiled and clean linens, coats, and aprons sha..."


In [102]:
dff.to_csv("Violations_Parsed.csv",index=False)